In [ ]:
## importamos las librerías necesarias
import requests ## permite realizar solicitudes HTTP
from bs4 import BeautifulSoup ## crea la "sopa", es decir, extrae el código de HTML de la página para poder obtener los datos 
import re ## busca patrones de texto
import pandas as pd ##crea y manipula Data Frames

In [ ]:
### El siguiente código accede a la página de Transfermarkt que tiene el ranking FIFA del 9 de enero del 2023.
### Después, descarga los enlaces para el perfil de cada equipo y los modifica para poderlos utilizar en el siguiente código.
## Para finalizar, guarda los enlaces modificados, el nombre de la selección y el año de análisis en un mismo diccionario.

page= 'https://www.transfermarkt.mx/statistik/weltrangliste/statistik/stat/plus/0/galerie/0?datum=2023-01-09' ## Colocamos la página de las selecciones
headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'} ## esta línea agrega algunos "headers" que ayudan a evitar que las páginas bloqueen la petición 
page_tree = requests.get(page, headers=headers) ## pedimos la información de la página
soup = BeautifulSoup(page_tree.content, 'html.parser') ## creamos la "sopa"
table= soup.find('table', {'class':'items'}) ## encontramos la tabla de donde queremos obtener la información (esto ya es del código de html)
table_body= table.find('tbody') ## seleccionamos sólo el cuerpo de la tabla
tds= table_body.find_all('td', {'class':'hauptlink'}) ## seleccionamos los datos específicos que necesitamos (depende del html)
seleccionlink={'Selección':[],'Link':[], 'Year':[]} ## creamos un diccionario para guardar los datos
linksolo=[] ## creamos un vector en donde guardaremos los enlaces como están en la página para después manipularlos
tdsnecesarios=[]## creamos un vector que guarde solo los valores de las filas que necesitamos
seleccion=[]## vector que guarda los nombres de las selecciones
for y in range(0, len(tds), 2): 
    tdsnecesarios.append(y) ## guarda los números de las filas específicas que necesitamos
for j, te in enumerate(tds):
    for sele in range(0, len(tdsnecesarios)):
        if j==tdsnecesarios[sele]: ## comparamos el número de la fila con el vector que habíamos creado
            img=te.find('img') ## encontramos el nombre
            a=te.find('a') ## encontramos el enlace
            seleccion.append(img['alt']) ## guardamos el nombre
            linksolo.append(a['href']) ## guardamos el enlace
for l in range(0,len(seleccion)):
    for year in range(2009, 2022):
            y = str(year) ## guardamos el año
            seleccionlink['Selección'].append(seleccion[l]) ## guardamos el nombre del equipo en el diccionario
            seleccionlink['Link'].append('https://www.transfermarkt.mx' + linksolo[l] + '?saison_id=' + y) ## modificamos el enlace y lo guardamos en el diccionario
            seleccionlink['Year'].append(y) ## guardamos el año en el diccionario

In [ ]:
## El siguiente código accede al perfil de cada equipo nacional en los años que analizo. Después, obtiene el nombre, posición,
## enlace al perfil, lugar de nacimiento (país), fecha de debut en el equipo, edad y su nombre completo. Posteriormente,
## modifica el enlace de tal manera que muestre su rendimiento (este es el enlace que muestra las ligas en las que ha jugado a 
## lo largo de su carrera) y obtiene la liga en la que jugó, la temporada y los minutos.

jugadorcompleto={'Name':[],'UniqueName':[],'BirthPlace':[],'FinalNationalTeam':[], 'Debut':[],'NationalTeam':[],'Year':[],
                 'Season':[],'League':[],'Age':[],'Minutes':[],'Position':[]} ## creamos el diccionario para los datos (finales) que necesitamos
for i in range(0,n):## este número se modifica de acuerdo al equipo y año de la que queremos que obtenga los datos
    selección = seleccionlink['Selección'][i]## definimos el nombre del equipo de acuerdo con el índice del diccionario del código anterior 
    año = int(seleccionlink['Year'][i])+1 ## definimos el año de acuerdo con el índice del diccionario del código anterior +1 
    page = seleccionlink['Link'][i] ## definimos el enlace de acuerdo con el índice del diccionario del código anterior
    headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'} ## esta línea y las que siguen son idénticas a las líneas del código anterior
    page_tree = requests.get(page, headers=headers)
    soup = BeautifulSoup(page_tree.content, 'html.parser')
    table= soup.find('table', {'class':'items'})
    table_body= table.find('tbody')
    tds= table_body.find_all('td', {'class':'zentriert'})
    ## NOTA: La mayoría del código siguiente depende de la página, tome en cuenta que este código está diseñado específicamente
    ## para Transfermarkt, si necesita una mayor explicación contacte directamente al autor
    inline_tables=table_body.find_all('table', {'class':'inline-table'})
    clubs=[]
    ages=[]
    for i in range(1, len(tds),3):
        ages.append(i)
    nameypos={'Name':[],'Position':[],'Links':[],'Linksrend':[],'LugardeNac':[],'Debut':[],
              'Selec':[],'Age':[],'NombreID':[]} ## creamos un diccionario intermedio para guardar los datos que podemos obtener a partir de los primeros enlaces y que guarda los enlaces modificados
    for x in inline_tables:
        player = x.find('img')
        nameypos['Name'].append(player['alt'])
    for jk in inline_tables:
        tds2=jk.find_all('td')
        nameypos['Position'].append(tds2[2].text)
    getlink=[]
    for row in inline_tables:
        player_link = row.find('td', {'class':'hauptlink'})
        getlink.append(player_link)
    profile=[]
    for l in getlink:
        href=l.find('a')
        profile.append(href['href'])
    for h in profile:
        if h is not None:
            nameypos['Links'].append('https://www.transfermarkt.mx'+ h)
        else:
            nameypos['Links'].append('No Link')
    for link in nameypos['Links']:
        perfil=link
        rend= perfil.replace('profil', 'detaillierteleistungsdaten')
        nameypos['Linksrend'].append(rend)
    needed=[0,2]
    for i in range(0, len(nameypos['Name'])):
        page = nameypos['Links'][i]
        headers = {'User-Agent' : 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/47.0.2526.106 Safari537.36'}
        page_tree = requests.get(page, headers=headers)
        soup = BeautifulSoup(page_tree.content, 'html.parser')
        info=soup.find('div', {'class':'data-header__details'})
        items=info.find_all('ul')
        fichatec=soup.find('div',{'class':'info-table info-table--right-space'})
        if fichatec is not None:
            span=fichatec.find_all('span',{'class':'info-table__content info-table__content--bold'})
            nombrecomp=span[0].text
            nameypos['NombreID'].append(nombrecomp)
        else:
            nameypos['NombreID'].append('NA')
        lis=[]
        for g in items:
            dataheaders=g.find_all("li")
            lis.append(dataheaders)
        necesario=[]
        for f,s in enumerate(lis):
            for t in range(0, len(needed)):
                if f == needed[t]:
                    necesario.append(s)
        if len(necesario[0])>=2:
            nac=necesario[0][1].find('img')
            if nac is not None:
                nameypos['LugardeNac'].append(nac['alt'])
            else: 
                nameypos['LugardeNac'].append('Revisar')
        else:
            nameypos['LugardeNac'].append('Revisar')
        if len(necesario[1])!=0:
            sel=necesario[1][0].find('img')
            nameypos['Selec'].append(sel['alt'])
            carsels=soup.find('div',{'data-viewport':'Laenderspielkarriere'})
            fecha=carsels.find_all('div',{'class':'grid__cell grid__cell--center'})
            ade=fecha[0].find('a')
            if ade is not None:
                debut=ade.text
                nameypos['Debut'].append(debut)
            else:
                debut=fecha[0].text
                debutcl=debut.replace("\n", "").replace("\t", "").replace(" ", "")
                nameypos['Debut'].append(debutcl)
        else:
            nameypos['Selec'].append('NA')
            nameypos['Debut'].append('NA')
        for j, row in enumerate(tds):
            if j in ages:
                    age=row.text
                    if age is not None:
                        nameypos['Age'].append(age)
                    else:
                        nameypos['Age'].append('0')
    for i in range(0, len(nameypos['Name'])):
        posicion=nameypos['Position'][i]
        nombreid=nameypos['NombreID'][i]
        edad=nameypos['Age'][i]
        seleccionquerepresenta=nameypos['Selec'][i]
        debutsele=nameypos['Debut'][i]
        nacimiento=nameypos['LugardeNac'][i]
        nombre=nameypos['Name'][i]
        page_rend = nameypos['Linksrend'][i]
        pagerend_tree = requests.get(page_rend, headers=headers)
        soup_rend = BeautifulSoup(pagerend_tree.content, 'html.parser')
        table = soup_rend.find('div', {'class': 'responsive-table'})
        body = table.find('tbody')
        if body is not None:
            temp = body.find_all('td', {'class': 'zentriert'})
            comp = body.find_all('td', {'class': 'hauptlink no-border-links'})
            mins = body.find_all('td', {'class': 'rechts'})
            tempn = []
            if nameypos['Position'][i] == 'Portero':
                for y in range(0, len(temp), 6):
                    tempn.append(y)
            else:
                for y in range(0, len(temp), 8):
                    tempn.append(y)
            for j, te in enumerate(temp):
                for h in range(0, len(tempn)):
                    if j == tempn[h]:
                        im = te.text
                        jugadorcompleto['Position'].append(posicion)
                        jugadorcompleto['Age'].append(edad)
                        jugadorcompleto['FinalNationalTeam'].append(seleccionquerepresenta)
                        jugadorcompleto['Debut'].append(debutsele)
                        jugadorcompleto['BirthPlace'].append(nacimiento)
                        jugadorcompleto['UniqueName'].append(nombreid)
                        jugadorcompleto['Name'].append(nombre)
                        jugadorcompleto['NationalTeam'].append(selección)
                        jugadorcompleto['Year'].append(año)
                        jugadorcompleto['Season'].append(im)
            for z in comp:
                lig=z.find('a')
                jugadorcompleto['League'].append(lig.text)
            for w in mins:
                text = w.text
                mj = ''.join(caracter for caracter in text if caracter.isdigit() or caracter == '.')
                mj_f = mj.replace('.', '')
                if mj_f == '':
                    jugadorcompleto['Minutes'].append(int('1'))
                else:
                    jugadorcompleto['Minutes'].append(int(mj_f))
        else:
            jugadorcompleto['Position'].append(posicion)
            jugadorcompleto['Age'].append(edad)
            jugadorcompleto['FinalNationalTeam'].append(seleccionquerepresenta)
            jugadorcompleto['Debut'].append(debutsele)
            jugadorcompleto['BirthPlace'].append(nacimiento)
            jugadorcompleto['UniqueName'].append(nombreid)
            jugadorcompleto['NationalTeam'].append(selección)
            jugadorcompleto['Year'].append(año)
            jugadorcompleto['Name'].append(nombre)
            jugadorcompleto['Season'].append('NA')
            jugadorcompleto['League'].append('NA')
            jugadorcompleto['Minutes'].append('NA')
## NOTA: algunos jugadores y equipos tienen páginas construidas de diferente forma. Para estos, rellenamos el diccionario con 'NA'
## lo que nos permite crear el Data Frame sin errores. Estos datos fueron corroborados manualmente y corregidos en caso de que fuera
## necesario.

In [ ]:
players=pd.DataFrame(jugadorcompleto) ## creamos un data frame a partir del diccionario

In [ ]:
## Convertimos el data frame en excel y lo guardamos
players.to_excel('C:/Nombre.xlsx', encoding='utf-8')